<a href="https://colab.research.google.com/github/OliveiraMatheuss/Automacao_Rampage/blob/main/automacao_comprovantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Leitor de comprovantes bancários 

## Iniciar Script

In [ ]:

# Instalações
!sudo apt-get install tesseract-ocr-por | grep -v 'already satisfied'
!pip install pytesseract  | grep -v 'already satisfied'
!pip install gspread_dataframe==3.3.0  | grep -v 'already satisfied'
!pip install gspread==5.4.0  | grep -v 'already satisfied'


In [28]:

#------Importações de Pacotes -----------------------------------------------------------------------------------|

import pytesseract
import pandas as pd
import re
from tqdm import tqdm
import glob
from PIL import Image
from oauth2client.client import GoogleCredentials 
from google.auth import default
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe

#------------------------------------------------------------------------------------------------------------------------#

#------------------------------------------------------------------------------------------------------------------------|

# Montar o drive para buscar os arquivos
from google.colab import drive
drive.mount('/content/drive') #Acessar drive da Rampage

#------------------------------------------------------------------------------------------------------------------|
# Inicio Função de leitura dos comprovantes -----------------------------------------------------------------------------|

def tabela_pix():
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    print('Cole o Caminho onde estão os comprovantes\n')
    path = input()
    diretorio = glob.glob(path + '/*.jpg')

    print('\n Digite o nome da tabela \n')
    name_tab = input()

    arquivo = []
    entrada = []
    saida = []
    data_transacao = []
    descricao = []
    pagador = []

    for nome in tqdm(diretorio):
        
        #Leitura da imagem
        diretorio = nome
        image_path_in_colab = nome

        #Extrai a informaçõe em formato de texto
        extractedInformation = pytesseract.image_to_string(Image.open(image_path_in_colab),lang="por")
        texto = extractedInformation
        manipula_arquivo = nome.split('/')
        manipula_arquivo = manipula_arquivo[-1] # Pega somente o nome do arquivo e ignora o diretório
        arquivo.append(manipula_arquivo) # guarda o nome do arquivo em uma lista
            
        # Buscar  valor Recebido
        fatia = texto.split() 

       

        try:
          #Tratativa de erro, na leitura do comproavante pode haver comprovantes com R$ ou com RS 
          
          indice_RS = fatia.index('RS')
        except ValueError:
          indice_RS = fatia.index('R$')


        # Encontrar os tipos de entrada( receber uma transferência)
        if (('Pixrecebido' in texto) 
              or ('Pix recebido' in texto) 
              or ('Transferência recebida' in texto) 
              or ('Cashback Promocional' in texto) 
              or ('Cashback' in texto) 
              or ('Boleto recebido 48 horas' in texto) 
              or ('Transferência recebida' in texto)
                  ):  
            
            entrada.append(fatia[indice_RS+1]) # Valor do Pix Recebido
            saida.append(' ') # vazio pois é uma entrada
            fatia = texto.split('\n')
            fatia = [elemento for elemento in fatia if elemento.strip() != ""]
            if 'Descrição' in texto:
              
                
              if 'Descrição' in fatia[fatia.index('Quem enviou')-1]:
                    
                  descricao.append(fatia[fatia.index('Quem enviou')-1][9:])
                
              else: 
                 descricao.append(fatia[fatia.index('Quem enviou')-1])
            else: 
              descricao.append('sem descricao')

            
        # Encontrar os tifpo de saida 
        elif (('—Compra-crédito debitado' in texto) 
                or ('Pix enviado' in texto) 
                or ('Pix enviado devolvido' in texto) 
                or ('Pagamento efetuado' in texto) 
                or ('Compra no débito' in texto) 
                or ('Compra - crédito debitado' in texto) 
                or ('Transferência enviada' in texto)
                or ('me Boleto recebido 48 horas')
                    ):
          
            entrada.append(' ')
            saida.append(fatia[indice_RS+1])

            fatia = texto.split('\n')
            fatia = [elemento for elemento in fatia if elemento.strip() != ""]

            if 'Descrição' in texto:


                  if 'Descrição' in fatia[fatia.index('Quem recebeu')-1]:
                    
                      descricao.append(fatia[fatia.index('Quem recebeu')-1][9:])
                
                  else: 
                    descricao.append(fatia[fatia.index('Quem recebeu')-1])

            else: 
                descricao.append('sem descricao')

        # Se o tipo de transferência não foi especificado aqui
        else:
              entrada.append('tipo de pagamento não suportado')
              saida.append('tipo de pagamento não suportado')
            
        #Buscar a data do Pagamento
        matchs = re.search(r'(\d+/\d+/\d+)', texto)
        data_transacao.append(matchs.group(1)) # Data
            
        # Buscar nome do Pagador
        fatia = texto.split('\n')
        fatia = [elemento for elemento in fatia if elemento.strip() != ""]
        print(fatia) #REMOVER

        if ('Cashback Promocional' in texto) or ('Cashback' in texto):
          pagador.append('Cashback')
        elif ('Boleto recebido 48 horas' in texto) or ('me Boleto recebido 48' in texto):
          pagador.append('Boleto recebido')
        elif 'Outras saídas (2' in texto:
          pagador.append(fatia[fatia.index('Outras saídas (2')-1])

        else:
           pagador.append(fatia[fatia.index('Outras entradas (2')-1])
        
            
      


    a = {'Nome':pagador, 'data': data_transacao, 'entrada': entrada, 'Saida':saida, 'Descrição': descricao, 'arquivo':arquivo}
    df = pd.DataFrame.from_dict(a, orient='index') 
    df = df.transpose()

    #Cria a planilha que vai receber os dados
    """
     para pegar o folder id : abra a pasta no drive e copia a ultima parte do link 
     https://drive.google.com/drive/u/0/folders/<folder_id> 

    """

    folder_id = '1ND0LulP2nnHGGC-QRRV34uSt5Hwt1qnR'
    gc.create(name_tab, folder_id= folder_id)
    
    #Abre a planilha via Python
    gs = gc.open(name_tab)
    worksheet1 = gs.worksheet('Página1') # Nome da pagina padrão ao criar uma planilha
    worksheet1.clear()
    set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False)

    return df

# Fim da Função -------------------------------------------------------------------------------------------------------

# Inicio das entradas ------------------------------------------------------------------------------------------------


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Gerar Tabela

/content/drive/MyDrive/Dados/Gestao/Financeiro/2023/test

In [ ]:
df = tabela_pix()

Cole o Caminho onde estão os comprovantes

/content/drive/MyDrive/Dados/Gestao/Financeiro/2023/Prints_Abril

 Digite o nome da tabela 

caixa_abril_2023


  1%|          | 1/173 [00:02<07:02,  2.46s/it]

['Rr', 'e Pix recebido', 'E', 'R$ 16,00', 'Neide Aparecida Fernandes', 'Outras entradas (2', 'Sobre a transação', 'Data da transação', 'Segunda-feira, 03/04/2023', 'Horário 12h53', 'ID da transação', 'E60701190202304031552DY5VKMWL Ô', 'VS7', 'Descrição 1 vendinha', 'Quem enviou', 'Nome Neide Aparecida Fernandes', 'CPF/CNPJ ***366.256-**', 'III O «']


  1%|          | 2/173 [00:03<05:28,  1.92s/it]

['Rr', 'e Pix recebido', 'G', 'R$ 89,00', 'Thiago Vieira Silveira', 'Outras entradas (2', 'Sobre a transação', 'Data da transação', 'Segunda-feira, 03/04/2023', 'Horário 13h43', 'ID da transação', 'E18236120202304031641514305bbbd Ô', 'o', 'Quem enviou', 'Nome Thiago Vieira Silveira', 'CPF/CNPJ *** 538.526-**', 'Instituição NU PAGAMENTOS - IP', 'III O «']


...

 27%|██▋       | 46/173 [01:13<03:34,  1.69s/it]

['11:08', 'e Pix recebido', 'G', 'R$ 10,00', 'lara Lima Moura', 'Outras entradas (2', 'Sobre a transação', 'Data da transação', 'Quarta-feira, 12/04/2023', 'Horário 12h36', 'ID da transação', 'E00000000202304121536395849442 O', '04', 'Descrição Rocambole e musse lara', 'Quem enviou', 'Nome lara Lima Moura', 'CPF/CNPJ ***841.286-**', 'III O «']


 27%|██▋       | 47/173 [01:14<03:16,  1.56s/it]

['11:08', 'e Pix recebido', 'G', 'R$ 7,00', 'Isabella Costa Lacerda', 'Outras entradas (2', 'Sobre a transação', 'Data da transação', 'Quarta-feira, 12/04/2023', 'Horário 12h29', 'ID da transação', 'E86389236202304121529UZEwDWags Ô', 'Hg', 'Quem enviou', 'Nome Isabella Costa Lacerda', 'CPF/CNPJ *** 475.506-**', 'Instituição COOP SICOOB UNI SUDESTE', 'III O «']


 28%|██▊       | 48/173 [01:16<03:00,  1.45s/it]

In [30]:
df

,Nome,data,entrada,Saida,Descrição,arquivo
0,Guilherme De Almeida Paula,19/04/2023,,"69,58",compra eletrônica,Cópia de Screenshot_20230427-111849_Inter.jpg
1,Boleto recebido,25/04/2023,,"316,90",sem descricao,Cópia de Screenshot_20230427-112149_Inter.jpg
2,Guilherme Da Costa Oliveira,26/04/2023,"3,50",,sem descricao,Cópia de Screenshot_20230427-112221_Inter.jpg


## Contato

* Autor  : Matheus Ramos de Oliveira

* contato: ramos.matheus@engenharia.ufjf.br

* LinkedIn: https://www.linkedin.com/in/oliveiramatheuss/




